# Coursera Data Science Capstone Proj

##### This notebook will be used primarily for my final project in Data Science from the IBM course

In [2]:
import pandas as pd
import numpy as np

In [2]:
print("Hello capstone project course!")

Hello capstone project course!


### Toronto Neighborhoods Dataframe

Read in table from Wikipedia into dataframe

In [3]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
        

In [28]:
df1 = pd.read_html(wikiurl)
df1[0]
df = pd.DataFrame(df1[0])
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove rows where there is no borough assigned.

In [29]:
df=df[df.Borough != 'Not assigned']
df.shape

(103, 3)

In [30]:
df.reset_index(inplace=True)

In [31]:
df.drop(['index'], axis = 1, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [37]:
for i in range(df.shape[0]):
    if df["Neighborhood"][i] == np.nan:
        df["Neighborhood"][i] = df["Borough"][i]

In [43]:
df['Postal Code'].count()

103

In [46]:
df.shape

(103, 3)